In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
import os

warnings.filterwarnings('ignore')

# [Module 3] 결측치 보간 및 군집화 (Clustering)
# 목적: 기상 데이터의 연속성 확보 및 지역/기후 특성 추출

INPUT_PATH = 'temp_integrated_data.feather'#사전에 생성된 파일 경로

print("--- [Step 3] 보간 및 군집화 ---")

# 1. 데이터 로드 (이전 단계 결과물)
if os.path.exists(INPUT_PATH):
    df_full = pd.read_feather(INPUT_PATH)
    print(f"데이터 로드 완료. 총 행 수: {len(df_full)}")
else:
    # 파일이 없을 경우 안내 메시지
    raise FileNotFoundError(
        f"'{INPUT_PATH}' 파일을 찾을 수 없습니다.\n"
        "modules/02_preprocessing.ipynb 파일을 열고, 마지막 줄의 "
        "'df_full.to_feather(OUTPUT_PATH)' 주석(#)을 제거한 뒤 실행해주세요."
    )

# 2. 하이브리드 보간 (Hybrid Interpolation)
print("결측치 보간 수행...")
cols_weather = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure', 
    'real_feel_temp', 'real_feel_temp_shade', 'rel_hum', 'temp_b', 'uv_idx', 'vis', 
    'wind_chill_temp', 'wind_dir_b', 'wind_gust_spd', 'wind_spd_b', 'cloud_a', 
    'ground_press', 'humidity', 'rain', 'snow', 'temp_a', 'temp_max', 'temp_min', 
    'wind_dir_a', 'wind_spd_a'
]
cols_exist = [c for c in cols_weather if c in df_full.columns]

# 시계열 연속성을 위해 정렬 후 FFill/BFill 적용
df_full.sort_values(by=['pv_id', 'time'], inplace=True)
df_full[cols_exist] = df_full.groupby('pv_id')[cols_exist].ffill().bfill()

# 3. 이중 군집화 (Dual Clustering)
print("이중 군집화(Location + Climate) 모델링...")
scaler = StandardScaler()

# (1) 위치 군집 (Location Cluster) - K=20
# 지형적 특성 반영
loc_features = ['coord1', 'coord2']
scaled_loc = scaler.fit_transform(df_full[loc_features])
kmeans_loc = KMeans(n_clusters=20, random_state=42, n_init=10)
df_full['cluster_loc'] = kmeans_loc.fit_predict(scaled_loc)
df_full['cluster_loc'] = df_full['cluster_loc'].astype('category')

# (2) 기후 군집 (Climate Cluster) - K=10
# 거시적 기상 패턴 반영
clim_features = ['temp_a', 'humidity', 'wind_spd_a', 'pressure', 'vis']
target_climate = [c for c in clim_features if c in df_full.columns]
pv_stats = df_full.groupby('pv_id')[target_climate].mean()

scaled_cli = scaler.fit_transform(pv_stats)
kmeans_cli = KMeans(n_clusters=10, random_state=42, n_init=10)
pv_labels = kmeans_cli.fit_predict(scaled_cli)

# 매핑
climate_map = dict(zip(pv_stats.index, pv_labels))
df_full['cluster_climate'] = df_full['pv_id'].map(climate_map).astype('category')

print("모듈 처리 완료. (Main Pipeline으로 통합됨)")